## Compare old IITM LPS_track with newer IITM LPS_output files to see if they have changed

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

In [2]:
# !pip install nbimporter

## helpers

In [3]:
# "".join([str(int(x)).zfill(2) for x in list(group.iloc[0][2:5])]) outputs something of the form
#    yyyymmdd, e.g., '20070618'

def create_id(x):
    key = ("".join([str(int(elem)).zfill(2) for elem in list(x.iloc[0,2:5])]) # yyyymmdd
           +"-"
           +str(int(x.iloc[0,5])) # hh
           +"-"
           +"-".join([str(float(elem)) for elem in list(x.iloc[1][3:5])]) # lon-lat
          )
    return key

In [4]:
# There is 1 empty column at the start of all rows in LPS_track files (it is where
#    the 'start' in the 'start 41 2007 06 18 00' lines goes)

# param: csv: a text file (of the form LPS_track_yyyymmdd.txt) supplied by Vishnu
# param: column_names: a list of column names
# returns: a dictionary whose keys are dates of storms and values are dataframes
def read_vishnu_csv_to_dic(csv, column_names, data_name, keyIncludesHour=False, 
                           keyToCheckForHit=None, obsToCheckForHit=None,
                           lonMin=50, lonMax=100, latMin=0, latMax = 30, yearMin=2007, yearMax=2018,
                           monthMin=6, monthMax=9):
    
    file = csv
    df = pd.read_csv(file, header=None, names=column_names, sep='\t')
    
    # df[0] gets the first column, which is empty except for 'start'
    # .isin(['start'] returns a boolean array, where it is True wherever there is 'start' and False
    #    otherwise
    # .cumsum() returns an array of numbers, that starts at the first True and takes the value 1 at
    #    every index until the next True, after which it takes the value 2 until the next True, and
    #    so on.
    # so, the dataframe is grouped into the distinct storms, based on whenever there is a 'start' line.
    groups = df[0].isin(['start']).cumsum()
    
    # group.iloc[1:, 1:] gets rid of the first row (with 'start') and the first column (that is empty
    #    in all the rows except for in the 'start' row
    # so, in dic_of_dfs, the keys are the unique IDs based on genesis time and position
    #    (like '20070618-12-87.5-17.5') and the values are dataframes
    dic_of_dfs = {create_id(group): 
                  pd.concat([group.iloc[1:, 1:],
                             pd.Series(create_id(group),
                                       index=group.iloc[1:].index).rename(data_name+' key')], 
                            axis=1) 
                  for name, group in df.groupby(groups)
                  if (lonMin <= group.iloc[1, 3] <= lonMax
                      and latMin <= group.iloc[1, 4] <= latMax
                      and yearMin <= group.iloc[0, 2] <= yearMax
                      and monthMin <= group.iloc[0, 3] <= monthMax)}
        # the if condition filters for storms that are over India,
        #    and between 2007 and 2018,
        #    and in the Indian summer monsoon season (JJAS: June, July, August, September)
        # for 'over India', need longitude between 50 and 100 degrees, and latitude 
        #    between 5 and 30 degrees.
        
        # for reference:
        # group.iloc[1:]['Longitude'].between(lonMin, lonMax).all() 
        # and group.iloc[1:]['Latitude'].between(latMin, latMax).all()
    
    return dic_of_dfs

In [5]:
# param: df: a pandas dataframe containing columns titled 'Year', 'Month', 'Day', 'Hour'
# returns: a datafram identical to df, except containing an extra column 'date'
def add_datetime_to_df(df):
    
    # the .to_datetime method relies on the columns having sensible names 'Year', 'Month', etc.
    df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])
    
    return df

# param: dic: a dictionary whose values are dataframes
# returns: a dictionary with all the dataframes having an extra 'date' column
def add_datetime_to_dic(dic):
    
    return {key: add_datetime_to_df(value) for key, value in dic.items()}

## code

In [6]:
IITM_new = {}
nempty_iitm = 0
ntotal_iitm = 0
empty_dates_new = []

column_names = [0, 'XGridCentre', 'YGridCentre', 'Longitude', 'Latitude', 'MinSF', 'MinSLP', 'AvgRH', 
                'MaxSfcGeoPt', 'MaxSfcWind', 'minmslix', 'Year', 'Month', 'Day', 'Hour']

folder_path = '/global/homes/s/salilg/pandasNewCode/tempest_extremes/iitm_output_files_salil/'
for outputfile in sorted(os.listdir(folder_path)):
    if outputfile.startswith('LPS_dslp_output'):
        
        date = outputfile[16:24]
        #all IITM file dates are observed date + 1; so moving back one day here to match with ERA5 file dates
        # date = changeIITMFileDate(fileDate)
        
        file = folder_path + outputfile
        
        if os.stat(file).st_size == 0:
            nempty_iitm += 1
            empty_dates_new.append(date)
        
        else:
            
            dic = add_datetime_to_dic(
                read_vishnu_csv_to_dic(file, column_names, 'IITM')
                                            )
            IITM_new[date] = dic # this risks skipping dates that are duplicated, but
            # not important for now
            
            # load_model_data(date, possible_model_tracks, IITM, ERA5, 'ERA5')
            
        ntotal_iitm += 1

In [7]:
nempty_iitm

11

In [8]:
IITM_old = {}
# IITM_multiple_hits = {}

column_names = [0, 'XGridCentre', 'YGridCentre', 'Longitude', 'Latitude', 
                    'MinSF', 'MinSLP', 'PressureDrop', 'MaxSfcWind', 'AvgRH', 'MaxSfcGeoPt', 'Year', 
                    'Month', 'Day', 'Hour']

for outputfile in sorted(os.listdir('/global/cscratch1/sd/vishnus/GFS_IITM/2007_2018_newgp/')):
    if (outputfile.startswith('LPS_dslp_track_')): # and 
        # os.stat('/global/cscratch1/sd/vishnus/GFS_IITM/2007_2018_newgp/' + outputfile).st_size > 0):
        
        date = outputfile[15:23]
        # all IITM file dates are observed date + 1; so moving back one day here
        # date = changeIITMFileDate(fileDate)
        
        # if date in common_hits:
        
        file = '/global/cscratch1/sd/vishnus/GFS_IITM/2007_2018_newgp/' + outputfile
            
        dic = add_datetime_to_dic(
            read_vishnu_csv_to_dic(file, column_names, 'IITM')
                                        )
        IITM_old[date] = dic # this risks skipping dates that are duplicated, but
        # not important for now

        # load_model_data(date, possible_model_tracks, IITM, ERA5, 'ERA5')
            
            # get_num_multiple_hits(date, possible_model_tracks, ERA5, IITM_multiple_hits)

In [9]:
len(IITM_new), len(IITM_old)

(147, 102)

In [10]:
common = [x for x in IITM_old.keys() if x in IITM_new.keys()]
len(common)

88

In [11]:
missing_from_new = [key for key in IITM_old.keys() if key not in IITM_new.keys()]
len(empty_dates_new), len(missing_from_new)

(11, 14)

In [13]:
# it looks like some of the dates (all 2011) in the new candidate files are off by one day.
# for example, there's a 20110602, 20110613, 20110704, 20110718, 20110807, 20110813, 20110818,
# 20110829, 20110904, 20110918
# but there are also some 2011 dates that are included (20110607, etc.)

[x for x in missing_from_new if x not in empty_dates_new]

['20110603',
 '20110614',
 '20110705',
 '20110719',
 '20110808',
 '20110814',
 '20110819',
 '20110830',
 '20110905',
 '20110919']

In [48]:
from datetime import timedelta

In [49]:
IITM_new_same_length = {}
for date in IITM_new:
    if date in IITM_old:
        IITM_new_same_length[date] = IITM_new[date]
    elif date in empty_dates_new:
        continue
    else:
        new = datetime(int(date[:4]), int(date[4:6]), int(date[6:8])) + timedelta(1)
        newdate = str(new.year).zfill(4) + str(new.month).zfill(2) + str(new.day).zfill(2)
        if newdate in IITM_old:
            IITM_new_same_length[newdate] = IITM_new[date]
        else:
            continue

In [50]:
len(IITM_new_same_length)

98

## compare all the common dates and see if dataframes are exactly the same

In [44]:
n = 0
k = 0
diff, same = 0, 0
for date in common:
    # check if, for a given date, both files have the same number of tracks
    if len(IITM_new[date]) != len(IITM_old[date]):
        n += 1
    else:
        for track_new, track_old in zip(IITM_new[date], IITM_old[date]):
            # if, for a given date, both files have the same number of tracks, then
            # the sub-dictionaries (IITM_new[date] and IITM_old[date]) should be of the 
            # same length, and moreover the keys for each track should be the same
            if track_new != track_old: # compare the keys
                k += 1
            else:
                # print(date, track_new, track_old)
                df_new = IITM_new[date][track_new].drop(columns=['MaxSfcGeoPt',
                                                                 'minmslix']).sort_index(axis=1)
                df_old = IITM_old[date][track_old].drop(columns=['MaxSfcGeoPt',
                                                                 'PressureDrop']).sort_index(axis=1)
                if df_new.equals(df_old):
                    same += 1
                else:
                    diff += 1
print(n, k, same, diff)

24 0 115 0


In [52]:
n = 0
m = 0
diff, same = 0, 0
for date in [x for x in IITM_new_same_length.keys() if x in IITM_old.keys()]:
    # check if, for a given date, both files have the same number of tracks
    if len(IITM_new_same_length[date]) != len(IITM_old[date]):
        n += 1
    else:
        k = 0
        for track_new, track_old in zip(IITM_new_same_length[date], IITM_old[date]):
            # if, for a given date, both files have the same number of tracks, then
            # the sub-dictionaries (IITM_new[date] and IITM_old[date]) should be of the 
            # same length, and moreover the keys for each track should be the same
            if track_new != track_old: # compare the keys
                k += 1
            else:
                # print(date, track_new, track_old)
                df_new = IITM_new_same_length[date][track_new].drop(columns=['MaxSfcGeoPt',
                                                                 'minmslix']).sort_index(axis=1)
                df_old = IITM_old[date][track_old].drop(columns=['MaxSfcGeoPt',
                                                                 'PressureDrop']).sort_index(axis=1)
                if df_new.equals(df_old):
                    same += 1
                else:
                    diff += 1
        if k > 0:
            m += 1
print(n, m, same, diff)

30 4 115 0


In [37]:
df_new = IITM_new['20180916']['20180916-0-88.75-14.25']
df_old = IITM_old['20180916']['20180916-0-88.75-14.25']

In [19]:
# from LPS_output file, without nodefileeditor
IITM_new['20070624'].keys()

dict_keys(['20070624-0-66.875-20.5', '20070629-3-87.75-21.625'])

In [20]:
# from LPS_track file, after nodefileeditor
IITM_old['20070624'].keys()

dict_keys(['20070624-0-66.875-20.5', '20070626-9-85.0-17.375', '20070629-3-87.75-21.625'])

## run tempestextremes on ECMWF again with the IITM conditions (minelength 9, maxgap 4) and see if the ECMWF output files are different from the existing track files

### rather, we already have old output files. So can also compare the new ones against those.

In [23]:
ECMWF_old_track = {}

column_names = [0, 'XGridCentre', 'YGridCentre', 'Longitude', 'Latitude', 
                    'MinSF', 'MinSLP', 'PressureDrop', 'MaxSfcWind', 'AvgRH', 'MaxSfcGeoPt', 'Year', 
                    'Month', 'Day', 'Hour']

years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

for year in years:
    for outputfile in sorted(os.listdir('/global/project/projectdirs/m3310/salilg/' + year)):
        if (outputfile.startswith('LPS_track_')):

            date = outputfile[10:18]

            file = '/global/project/projectdirs/m3310/salilg/' + year + '/' + outputfile

            dic = add_datetime_to_dic(
                read_vishnu_csv_to_dic(file, column_names, 'ECMWF')
                                            )
            ECMWF_old_track[date] = dic # this risks skipping dates that are duplicated, but
            # not important for now

            # load_model_data(date, possible_model_tracks, IITM, ERA5, 'ERA5')

In [44]:
ECMWF_old_output = {}

column_names = [0, 'XGridCentre', 'YGridCentre', 'Longitude', 'Latitude', 'MinSF', 'MinSLP', 'AvgRH', 
                'MaxSfcGeoPt', 'MaxSfcWind', 'minmslix', 'Year', 'Month', 'Day', 'Hour']

years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

for year in years:
    for outputfile in sorted(os.listdir('/global/project/projectdirs/m3310/salilg/' + year)):
        if (outputfile.startswith('LPS_output_')):

            date = outputfile[11:19]

            file = '/global/project/projectdirs/m3310/salilg/' + year + '/' + outputfile

            dic = add_datetime_to_dic(
                read_vishnu_csv_to_dic(file, column_names, 'ECMWF')
                                            )
            ECMWF_old_output[date] = dic # this risks skipping dates that are duplicated, but
            # not important for now

In [45]:
len(ECMWF_old_track), len(ECMWF_old_output)

(135, 135)

In [38]:
ECMWF_new_output = {}

column_names = [0, 'XGridCentre', 'YGridCentre', 'Longitude', 'Latitude', 'MinSF', 'MinSLP', 'AvgRH', 
                'MaxSfcGeoPt', 'MaxSfcWind', 'minmslix', 'Year', 'Month', 'Day', 'Hour']


for outputfile in sorted(os.listdir('/global/homes/s/salilg/pandasNewCode/tempest_extremes/\
ecmwf_test_output_files_salil/')):
    if (outputfile.startswith('LPS_output_')):

        date = outputfile[11:19]

        file = '/global/homes/s/salilg/pandasNewCode/tempest_extremes/\
ecmwf_test_output_files_salil/' + outputfile

        dic = add_datetime_to_dic(
            read_vishnu_csv_to_dic(file, column_names, 'ECMWF')
                                        )
        ECMWF_new_output[date] = dic # this risks skipping dates that are duplicated, but
        # not important for now

In [39]:
# this includes 2019 files. Only going till 2018 will get us 135 files.
len(ECMWF_new_output)

149

### compare the common dates

In [40]:
len([x for x in ECMWF_new_output.keys() if x in ECMWF_old_track.keys()])

135

In [48]:
# dic_1 must have output files
# dic_2 must have track files
dic_1 = ECMWF_old_output
dic_2 = ECMWF_old_track

n = 0
m = 0
diff, same = 0, 0
for date in [x for x in dic_1.keys() if x in dic_2.keys()]:
    # check if, for a given date, both files have the same number of tracks
    if len(dic_1[date]) != len(dic_2[date]):
        n += 1
    else:
        k = 0
        for track_new, track_old in zip(dic_1[date], dic_2[date]):
            # if, for a given date, both files have the same number of tracks, then
            # the sub-dictionaries (IITM_new[date] and IITM_old[date]) should be of the 
            # same length, and moreover the keys for each track should be the same
            if track_new != track_old: # compare the keys
                k += 1
            else:
                # print(date, track_new, track_old)
                df_new = dic_1[date][track_new].drop(columns=['minmslix']).sort_index(axis=1)
                df_old = dic_2[date][track_old].drop(columns=['PressureDrop']).sort_index(axis=1)
                if df_new.equals(df_old):
                    same += 1
                else:
                    diff += 1
        if k > 0:
            m += 1
print(n, m, same, diff)

0 0 255 0


In [49]:
# dic_1 must have output files
# dic_2 must have track files
dic_1 = ECMWF_new_output
dic_2 = ECMWF_old_track

n = 0
m = 0
diff_nsystems = 0
for date in [x for x in dic_1.keys() if x in dic_2.keys()]:
    # check if, for a given date, both files have the same number of tracks
    if len(dic_1[date]) != len(dic_2[date]):
        n += 1
    else:
        k = 0
        same, diff = 0, 0
        for track_new, track_old in zip(dic_1[date], dic_2[date]):
            # if, for a given date, both files have the same number of tracks, then
            # the sub-dictionaries (IITM_new[date] and IITM_old[date]) should be of the 
            # same length, and moreover the keys for each track should be the same
            if track_new != track_old: # compare the keys
                k += 1
            else:
                # print(date, track_new, track_old)
                df_new = dic_1[date][track_new].drop(columns=['MaxSfcGeoPt',
                                                                 'minmslix']).sort_index(axis=1)
                df_old = dic_2[date][track_old].drop(columns=['MaxSfcGeoPt',
                                                                 'PressureDrop']).sort_index(axis=1)
                if df_new.equals(df_old):
                    same += 1
                else:
                    diff += 1
        if k > 0:
            m += 1
        if diff > 0:
            diff_nsystems += 1
            
print(n, m, diff_nsystems)

54 5 6


In [47]:
# dic_1 must have output files
# dic_2 must have track files
dic_1 = ECMWF_new_output
dic_2 = ECMWF_old_output

n = 0
m = 0
diff, same = 0, 0
for date in [x for x in dic_1.keys() if x in dic_2.keys()]:
    # check if, for a given date, both files have the same number of tracks
    if len(dic_1[date]) != len(dic_2[date]):
        n += 1
    else:
        k = 0
        for track_new, track_old in zip(dic_1[date], dic_2[date]):
            # if, for a given date, both files have the same number of tracks, then
            # the sub-dictionaries (IITM_new[date] and IITM_old[date]) should be of the 
            # same length, and moreover the keys for each track should be the same
            if track_new != track_old: # compare the keys
                k += 1
            else:
                # print(date, track_new, track_old)
                df_new = dic_1[date][track_new].drop(columns=['MaxSfcGeoPt']).sort_index(axis=1)
                df_old = dic_2[date][track_old].drop(columns=['MaxSfcGeoPt']).sort_index(axis=1)
                if df_new.equals(df_old):
                    same += 1
                else:
                    diff += 1
        if k > 0:
            m += 1
print(n, m, same, diff)

54 5 100 6


## do some manual analysis

In [23]:
dic_new = IITM_new[common[80]]
dic_old = IITM_old[common[80]]

In [24]:
dic_new.keys()

dict_keys(['20180720-0-89.25-18.625'])

In [25]:
dic_old.keys()

dict_keys(['20180720-0-89.25-18.625'])

In [26]:
np.mean([df['MaxSfcGeoPt'].mean() for df in dic_old.values()])

5656.589957416666

In [27]:
np.mean([df['MaxSfcGeoPt'].mean() for df in dic_new.values()])

6430.825318541666

In [28]:
total = np.array([])
for dic in IITM_new.values():
    for df in dic.values():
        total = np.concatenate((total, df['MaxSfcGeoPt'].to_numpy()))
        
np.mean(total)

5461.5027530916195

In [29]:
total = np.array([])
for dic in IITM_old.values():
    for df in dic.values():
        total = np.concatenate((total, df['MaxSfcGeoPt'].to_numpy()))
        
np.mean(total)

5144.131446476719

In [31]:
dic_old[list(dic_old.keys())[0]].head()

,XGridCentre,YGridCentre,Longitude,Latitude,MinSF,MinSLP,PressureDrop,MaxSfcWind,AvgRH,MaxSfcGeoPt,Year,Month,Day,Hour,IITM key,date
1,394,389,89.250,18.625,-22730310.0,99311.39,319.500,15.51504,91.67205,6.712623,2018.0,7.0,20.0,0.0,20180720-0-89.25-18.625,2018-07-20 00:00:00
2,394,392,89.250,19.000,-23440790.0,99354.31,241.750,17.63548,91.78888,6.712623,2018.0,7.0,20.0,3.0,20180720-0-89.25-18.625,2018-07-20 03:00:00
3,393,397,89.125,19.625,-23337360.0,99333.03,183.000,17.67655,92.01437,10.525010,2018.0,7.0,20.0,6.0,20180720-0-89.25-18.625,2018-07-20 06:00:00
4,386,399,88.250,19.875,-23334120.0,99165.12,217.125,16.71610,90.89626,630.744700,2018.0,7.0,20.0,9.0,20180720-0-89.25-18.625,2018-07-20 09:00:00
5,384,401,88.000,20.125,-23989960.0,99158.81,147.250,19.52704,90.71742,2035.791000,2018.0,7.0,20.0,12.0,20180720-0-89.25-18.625,2018-07-20 12:00:00


In [76]:
dic_new[list(dic_new.keys())[0]].head()

,XGridCentre,YGridCentre,Longitude,Latitude,MinSF,MinSLP,AvgRH,MaxSfcGeoPt,MaxSfcWind,minmslix,Year,Month,Day,Hour,IITM key,date
1,394,389,89.250,18.625,-22730310.0,99311.39,91.67205,36.74033,15.51504,254871.0,2018.0,7.0,20.0,0.0,20180720-0-89.25-18.625,2018-07-20 00:00:00
2,394,392,89.250,19.000,-23440790.0,99354.31,91.78888,36.74033,17.63548,256151.0,2018.0,7.0,20.0,3.0,20180720-0-89.25-18.625,2018-07-20 03:00:00
3,393,397,89.125,19.625,-23337360.0,99333.03,92.01437,36.74033,17.67655,257430.0,2018.0,7.0,20.0,6.0,20180720-0-89.25-18.625,2018-07-20 06:00:00
4,386,399,88.250,19.875,-23334120.0,99165.12,90.89626,259.42930,16.71610,258070.0,2018.0,7.0,20.0,9.0,20180720-0-89.25-18.625,2018-07-20 09:00:00
5,384,401,88.000,20.125,-23989960.0,99158.81,90.71742,1890.08700,19.52704,259349.0,2018.0,7.0,20.0,12.0,20180720-0-89.25-18.625,2018-07-20 12:00:00
